In [57]:
import solver as s
import numpy as np

M_0 = [[ -94.72819358,  434.46110037],
 [ 492.37137264,  -83.76364682],
 [-461.6632819 ,   12.30747059],
 [-234.53363339,  489.13163952],
 [ 259.29965552,  494.36247177]]
b_0 = [-182.72045151,    7.45832728,  284.61444311, -447.71059602,  -62.24412898]
M_1 = [[-371.11325071,   40.03482654,  196.47905434, -488.05364273,
   391.03383745],
 [ 130.40129912,    9.54504472,  226.46398193, -122.85122664,
   154.48380419]]
b_1 = [-229.5058073 ,  368.14576833]
M = [M_0, M_1]
b = [b_0, b_1]
M = [M_0, M_1]
b = [b_0, b_1]
feedForward(M, b,  [3.84121358738431e-10, 0.0890401485416654])

array([55906.58413349, 65071.23842458])

In [1]:
def ReLU(v):
    for i in range(len(v)):
        if v[i] < 0:
            v[i] = 0
    return v

def feedForward(M, b, v):
    for i in range(len(M)):
        if i == len(M)-1:
            v = np.dot(M[i], v)+b[i]
        else:
            v = ReLU(np.dot(M[i], v)+b[i])
        # print(v)
    return v

In [4]:
# random pick network parameters
def generate_network(d):
    res = []
    for i in range(len(d)-1):
        tmp = np.random.rand(d[i]*d[i+1]) * 1000 - 500
        res.append(tmp)
        tmp = np.random.rand(d[i+1]) * 1000 - 500
        res.append(tmp)
    return res

def generate_network_int(d):
    res = []
    for i in range(len(d)-1):
        tmp = np.random.randint(-10, 10, size=d[i]*d[i+1])
        res.append(tmp)
        tmp = np.random.randint(-10, 10, size=d[i+1])
        res.append(tmp)
    return res

In [41]:
d = [2,7,2]
t = generate_network(d)
#t = generate_network_int(d)
for a in t:
    # print(f"{np.array2string(a, separator=' ')}")
    cnt = 1
    for b in a:
        if (cnt == len(a)):
            print(b, end='\n')
        else:
            print(b, end=' ')
        cnt += 1

-405.02851617459237 356.8287527043974 382.56825854439853 440.32696579295566 -94.38514019516384 -412.50520889854306 233.43905000234327 -173.62471612888186 367.4115811766883 196.84618746381955 -388.39026636159036 -314.55629507540516 -367.95892859533376 425.0598190052017
-6.952326887242009 -499.1991004823677 -141.66366208191891 -239.1365833611714 54.2911242795185 -273.7425279165898 -18.062140660018656
377.53172800314917 -259.30876720869657 222.83111493356262 -125.92679465643744 -324.85794238116193 -410.27135841784093 487.1294413211501 211.3421673732846 278.9594348008667 -95.79779015475566 -356.31736788879164 152.97706375445898 316.2273610827856 -302.9236607579312
62.45127377063909 452.61791972800825


In [11]:
print(t)
for i in range(len(t)):
    for j in range(len(t[i])):
        print(t[i][j], end=' ')
        if (j+1)%d[i] == 0:
            print('\\', end='\n')
        else:
            print('&', end=' ')
    i += 1
    for j in range(len(t[i])):
        print(t[i][j], end=' ')
        if (j+1)%d[i] == 0:
            print('\\', end='\n')
        else:
            print('&', end=' ')

[array([ 7, -6,  3,  8,  8,  2,  9,  9,  2, -5,  6,  4, -1,  1, -9,  0]),
 array([ -1,   1,  -8,   3,   5,  -9, -10,  -9]),
 array([  7,   0,   8,   1,   3,  -9,   9,  -8,   9,  -3,  -6,  -3, -10,
          5,  -4,  -4]),
 array([-3,  5])]

In [55]:
from scipy.optimize import linprog
coeff_eq = [[1,0]]
coeff_in = [[1,-1]]
const_eq = [1]
const_in = [0]
PINF = float("inf");
NINF = float("-inf");
# used for transfer conditions with strict signs e.g. x < 1 -> x <= 1 - epsilon
epsilon = 0.00001
#print(coeff_eq, coeff_in, const_eq, const_in)
coeff_min = [0,1]
res_min = linprog(coeff_min,
              A_ub=coeff_in,
              b_ub=const_in,
              A_eq=coeff_eq,
              b_eq=const_eq,
              bounds = (NINF, PINF))

coeff_max = [0,-1]
res_max = linprog(coeff_max,
              A_ub=coeff_in,
              b_ub=const_in,
              A_eq=coeff_eq,
              b_eq=const_eq,
              bounds = (NINF, PINF))

interval = Interval(round(res_min.fun,6),round(-res_max.fun,6))
print(res_min, res_max)

ValueError: The algorithm terminated successfully and determined that the problem is infeasible.

In [85]:
def AND(e1, e2):
    '''
    logicAnd for inequalities
    
    @param e1, e2: inequalities in the form of [np.array([[]), ...]
    @return e: a new inequality system when both e1 and e2 are satisfied
    '''
    # Deal with true/false value
    if isinstance(e1,int):
        if e1 == 0:
            return 0
        elif e1 == 1:
            return e2
    if isinstance(e2,int):
        if e2 == 0:
            return 0
        elif e2 == 1:
            return e1
    # Combine each possible inequality matrix in two systems
    e = []
    if len(e1) == 0:
        return e2
    if len(e2) == 0:
        return e1
    for a in e1:
        for b in e2:
            e.append(np.r_[a,b])
    return e

def expand(e):
    res = []
    if isinstance(e, list) == False:
        return e
    else:
        for ele in e:
            tmp = []
            exp = []
            for element in ele:
                if (element[1] == 'N'):
                    exp.append([np.array([[element[0], '>', element[2]]], dtype=object), np.array([[element[0], '<', element[2]]], dtype=object)] )
                else:
                    tmp.append(element[:])
            if tmp == []:
                cur = []
            else:
                cur = [np.array(tmp, dtype=object)]
            for exponent in exp:
                cur = AND(cur, exponent)
            res += cur
        return res

expand([np.array([[1, 'N', 2],[2,'N',3],[3,'=',4]]), np.array([[1, '=', 2]])])

[array([['3', '=', '4'],
        ['1', '>', '2'],
        ['2', '>', '3']], dtype=object),
 array([['3', '=', '4'],
        ['1', '>', '2'],
        ['2', '<', '3']], dtype=object),
 array([['3', '=', '4'],
        ['1', '<', '2'],
        ['2', '>', '3']], dtype=object),
 array([['3', '=', '4'],
        ['1', '<', '2'],
        ['2', '<', '3']], dtype=object),
 array([['1', '=', '2']], dtype=object)]

In [48]:
from sympy import *
Set(1.0).inf

NotImplementedError: (Set(1.0))._inf

In [2]:
import solver as s
import numpy as np

M_0 = [[1, 2],
 [3,4],
 [5,6]]
b_0 = [1,2,3]
M_1 = [[1,1,1],
 [2,2,2]]
b_1 = [1,2]
M = [M_0, M_1]
b = [b_0, b_1]
M = [M_0, M_1]
b = [b_0, b_1]
feedForward(M, b,  [0.111111111703919,1.0000000044866])

array([20.00000006, 40.00000012])